## Semantic Search

In [75]:
from sentence_transformers import SentenceTransformer, InputExample, models, datasets, losses, util
import pandas as pd
import random
import os.path
from os import path
import time
import re
import pickle
from tqdm import tqdm

In [5]:
movie_data_path = '../movie_data.csv'
gen_queries_path = '../Queries/generated_queries.csv'

### Load in Movie Data

In [6]:
movie_data = pd.read_csv(movie_data_path,header=0)
movie_data

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


### Load in Generated Queries

In [4]:
queries_plots = pd.read_csv(gen_queries_path,header=0)

training_data = []
for _, row in queries_plots.iterrows():
    training_data.append(InputExample(texts=[row['Query'], row['Plot']]))

# shuffle training data
random.shuffle(training_data)

### Implement Search

In [57]:
# Load in the model
class SearchUsingBert():
    
    def __init__(self, data, training_data=None, base_model='sentence-transformers/msmarco-distilbert-base-dot-prod-v3', model_file_path='../Semantic Search/bert_models/search-bert-model', emb_file_path = '../Semantic Search/embeddings/plot_embeddings.pkl', finetune=True, num_of_epochs=3, save_model_path='../Semantic Search/bert_models/search-bert-model'):
        print("Initialising Bert Search Engine...\n")
        
        self.movie_data = data
        self.titles = list(data['Title'].to_numpy())
        self.plots = self.process_plot(list(data['Plot'].to_numpy()))

        # if model file exists, load in the model
        if path.exists(model_file_path):
            print("Model found, loading model from path...")
            self.model = self.load_model(model_file_path)
            print('Model loaded.\n')

        else:
            if training_data == None:
                print("Please provide training data.")
                return
            self.model = self.finetune_model(training_data, finetune, base_model, num_of_epochs)
            
        # if embeddings file exist, load in the embeddings
        if path.exists(emb_file_path):
            print("Pre-computed embeddings for movie plots found, loading from file...")
            
            with open(emb_file_path, "rb") as file:
                emb_data = pickle.load(file)
                self.plots_emb = emb_data['embeddings']
                
            print("Embeddings loaded.\n")

        else:
            print("Pre-computed embeddings for movie plots not found, encoding in progress...")
            self.plots_emb = self.model.encode(self.plots)
            
            # Save pre-computed plot embeddings to file for easier retrieval when testing
            with open(emb_file_path, "wb") as file:
                pickle.dump({'embeddings': self.plots_emb}, file)
                
            print("Embeddings generated.\n")

        print("Initialisation of Bert Search Engine completed.\n")

    def load_model(self, model_file_path=None):
        return SentenceTransformer(model_file_path)

    def process_plot(self, plots):
        processed_plots = []
        
        for plot in plots:
            processed_plot = re.sub(r'\[.*?\]', '', plot)
            processed_plots.append(processed_plot)
        
        return processed_plots

    def finetune_model(self, training_data, finetune, base_model, num_of_epochs, save_model_path):
        print("Model not found, default model will undergo finetuning...")

        if base_model != "":
            print("User-defined model: {}".format(base_model))
        else:
            print("Default model: {}".format(base_model))
        
        # Create sentence transformer model
        word_embeddings = models.Transformer(base_model)
        pooling = models.Pooling(word_embeddings.get_word_embedding_dimension())
        model = SentenceTransformer(modules=[word_embeddings, pooling])

        if finetune:
            # MultipleNegativesRankingLoss requires input pairs (query, relevant_passage)
            # and trains the model so that is is suitable for semantic search
            train_loss = losses.MultipleNegativesRankingLoss(model)

            # For the MultipleNegativesRankingLoss, it is important
            # that the batch does not contain duplicate entries, i.e.
            # no two equal queries and no two equal paragraphs.
            # To ensure this, we use a special data loader
            train_dataloader = datasets.NoDuplicatesDataLoader(training_data, batch_size=8)

            # Tune the model
            warmup_steps = int(len(train_dataloader) * num_of_epochs * 0.1)
            model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_of_epochs, warmup_steps=warmup_steps, show_progress_bar=True)

        # Save the model
        model.save(save_model_path)
        
        print('Model finetuning completed.\n')
        
        return model

    def cosine_sim(self, query, plots):
        scores = util.cos_sim(query, plots).tolist()[0]
        return scores
    
    def get_k_highest(self, k, scores):
        sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        return sorted_scores[0:k]

    def retrieve_movies(self, scores):
        results = []
        for idx, score in scores:
            results.append((self.movie_data.iloc[idx], score))
        
        return results
    
    def re_rank(self, query, scores, rerank_method='cross-encoder', encoder_name='cross-encoder/ms-marco-TinyBERT-L-6'):
        if rerank_method == 'cross-encoder':
            indexes = [i for i, _ in scores]
            inputs = [[query, self.plots[i]] for i in indexes]
            
            cross_encoder = CrossEncoder(encoder_name, max_length=512)
            scores = cross_encoder.predict(inputs)
            
            new_scores = sorted([(indexes[i], scores[i]) for i in range(len(indexes))], key=lambda x: x[1], reverse=True)
            return new_scores
    
    def search(self, query, top_k, print_progress=True, print_results=True, re_rank=False, rerank_method=None, cross_encoder=None):
        if print_progress:
            print('Searching in progress...')
        start_time = time.time()
        query_embedding = self.model.encode([query])
        sim_scores = self.cosine_sim(query_embedding, self.plots_emb)
        highest_k = self.get_k_highest(top_k, sim_scores)
        if re_rank:
            if rerank_method == None:
                print('Please specify a re-ranking method.')
                return
            if rerank_method == 'cross-encoder':
                if cross_encoder == None:
                    highest_k = self.re_rank(query, highest_k, rerank_method)
                else:
                    highest_k = self.re_rank(query, highest_k, rerank_method, cross_encoder)
        movies = self.retrieve_movies(highest_k)
        time_taken = time.time() - start_time
        if print_progress:
            print('Search completed.\n')

        results = {
            'results': movies,
            'query': query,
            'time': time_taken,
        }

        if print_results:
            self.print_results(results)
        
        return results

    def print_results(self, results):
        movies = results['results']
        print('-----------------Search Results-----------------')
        print('Total Search Time: {}'.format(results['time']))
        print('User Query: {}'.format(results['query']))
        print('Search Results:')
        for i in range(len(movies)):
            print('{}. {} ({}) ----- Score: {}'.format(i+1, movies[i][0]['Title'].strip(), movies[i][0]['Release Year'], movies[i][1]))
        print('\n')

#### Test 1: Using Base T5 Model for Query Generation

In [58]:
# Start up the search engine
search_engine = SearchUsingBert(movie_data)

Initialising Bert Search Engine...

Model found, loading model from path...
Model loaded.

Pre-computed embeddings for movie plots found, loading from file...
Embeddings loaded.

Initialisation of Bert Search Engine completed.



In [25]:
test_query = "spider man and his girlfriend"
k = 5

In [26]:
results = search_engine.search(test_query, k)

Searching in progress...

Search completed.

-----------------Search Results-----------------
Total Search Time: 0.11380505561828613

User Query: spiderman and his girlfriend

Search Results:
1. In the Nick (Similarity Score: 0.6592525839805603)
2. Spider (Similarity Score: 0.6130794882774353)
3. Spider-Man 2 (Similarity Score: 0.5729163885116577)
4. Earth vs. the Spider (Similarity Score: 0.5652084350585938)
5.  Earth vs. the Spider (Similarity Score: 0.5652084350585938)


##### With Re-ranking (Cross-Encoder)

In [ ]:
reranked_results = search_engine.search(test_query, k, re_rank = True, rerank_method='cross-encoder')

#### Test 2: Using T5 One Line Summary for Query Generation

In [59]:
# Start up the search engine
search_engine_2 = SearchUsingBert(movie_data, model_file_path="../Semantic Search/bert_models/search-bert-model-2", emb_file_path="../Semantic Search/embeddings/plot_embeddings_2.pkl")

Initialising Bert Search Engine...

Model found, loading model from path...
Model loaded.

Pre-computed embeddings for movie plots found, loading from file...
Embeddings loaded.

Initialisation of Bert Search Engine completed.



In [27]:
results_2 = search_engine_2.search(test_query, k)

Searching in progress...

Search completed.

-----------------Search Results-----------------
Total Search Time: 0.12586498260498047

User Query: spiderman and his girlfriend

Search Results:
1. Spider-Man 2 (Similarity Score: 0.6484066247940063)
2. Spider (Similarity Score: 0.6249087452888489)
3. Spun (Similarity Score: 0.5878573060035706)
4. Spider-Man 3 (Similarity Score: 0.5756661891937256)
5. In the Nick (Similarity Score: 0.5729314088821411)


##### With Re-ranking (Cross-Encoder)

In [ ]:
reranked_results_2 = search_engine_2.search(test_query, k, re_rank = True, rerank_method='cross-encoder')

#### Test 3: No Fine-Tuning

In [ ]:
# Start up the search engine
search_engine_3 = SearchUsingBert(movie_data, model_file_path="../Semantic Search/bert_models/search-base-bert-model", emb_file_path="../Semantic Search/embeddings/plot_embeddings_base.pkl", finetune=False)

In [ ]:
results_3 = search_engine_3.search(test_query, k)

##### With Re-ranking (Cross-Encoder)

In [ ]:
reranked_results_3 = search_engine_3.search(test_query, k, re_rank = True, rerank_method='cross-encoder')

In [ ]:
test_query = "girl follows a white rabbit down a hole"
k = 10
results = search_engine.search(test_query, k)
# Reranked


### Model Accuracy

In [69]:
test_queries = '../Semantic Search/test_queries/test_queries.csv'
test_queries_2 = '../Semantic Search/test_queries/test_queries_2.csv'

test_k = 1

titles = list(movie_data['Title'].to_numpy())

In [76]:
def test_accuracy(queries_plots, engine):
    total = 0
    correct = 0

    for idx in tqdm(range(len(queries_plots))): 
    # for idx, row in queries_plots.iterrows():
        query = queries_plots["Query"][idx]
        plot = queries_plots["Plot"][idx]
        correct_title = titles[idx]
        total += 1

        predicted_title = engine.search(query, test_k, print_progress=False, print_results=False)['results'][0][0]

        if predicted_title == correct_title:
            correct += 1
    
    accuracy = correct / total
    print("-----------------Accuracy Results-----------------")
    print("Total: {}".format(total))
    print("Correct: {}".format(correct))
    print("Accuracy: {}".format(accuracy))
    
    return accuracy


#### Test 1: Using Base T5 Model for Query Generation

In [73]:
test_queries_plots = pd.read_csv(test_queries,header=0)
test_queries_plots_2 = pd.read_csv(test_queries_2,header=0)

In [78]:
acc = test_accuracy(test_queries_plots, search_engine)

# test on the other test queries dataset
acc_other = test_accuracy(test_queries_plots_2, search_engine)

100%|██████████| 34886/34886 [1:10:47<00:00,  8.21it/s]


-----------------Accuracy Results-----------------
Total: 34886
Correct: 18162
Accuracy: 0.5206099868141948


100%|██████████| 34886/34886 [1:10:42<00:00,  8.22it/s]

-----------------Accuracy Results-----------------
Total: 34886
Correct: 19746
Accuracy: 0.5660150203520037


#### Test 2: Using T5 One Line Summary for Query Generation

In [79]:

acc_2 = test_accuracy(test_queries_plots_2, search_engine_2)

# test on the other test queries dataset
acc_2_other = test_accuracy(test_queries_plots, search_engine_2)

 99%|█████████▉| 34621/34886 [1:10:42<00:33,  7.87it/s]